In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))
suppressMessages(suppressWarnings(library(metap)))
suppressMessages(suppressWarnings(library(relaimpo)))

In [96]:
### Load hg19 gene,TSS information from 

In [6]:
gene_annotation<- read.table("data/gene_annot_jun10.txt",header=T)

In [7]:
head(gene_annotation,3)

,ENSGID,NAME,CHR,START,END,TSS
,<chr>,<chr>,<int>,<int>,<int>,<int>
1,ENSG00000186092,OR4F5,1,69091,70008,69091
2,ENSG00000235249,OR4F29,1,367640,368634,367640
3,ENSG00000185097,OR4F16,1,621059,622053,622053


In [113]:
# 3 Mb window around the lead variant with chromosome, start and stop

In [122]:
lead <- read.table("LEAD.txt",header=T)

In [124]:
# gene to variant mapping

In [125]:
gene_variant <- data.frame()

for (i in 1:nrow(trans_lead) ){
   
line_q <- trans_lead[i,]
marker_q <- as.character(line_q[1])
chr_q <- as.integer(line_q[2])
start_q<- as.integer(line_q[6])
end_q <- as.integer(line_q[7])
lead_q <- as.integer(line_q[5])
position_q <- as.integer(line_q[3])
tmp_df <- gene_annotation%>%filter(CHR==chr_q & START >= start_q & END <= end_q)%>%mutate(INDEX=marker_q,WINDOW=lead_q,POSITION=position_q,TSS_DISTANCE=abs(TSS-POSITION))
gene_variant <- rbind(gene_variant,tmp_df)
}

In [126]:
###Add MAGMA output 

In [127]:
magma <- read.table("MAGMA.genes.out",header=T)%>%rename(ENSGID=GENE)
gene_variant_1 <- merge(gene_variant,magma,by="ENSGID")%>%select(ENSGID,NAME,CHR.x,START.x,END,TSS,INDEX,WINDOW,POSITION,TSS_DISTANCE,P)%>%rename(CHR=CHR.x,START=START.x,MAGMA_P=P)

In [129]:
## Add coloc output 

In [130]:
coloc <- read.table("COLOC_H4.TAB",header=T)%>%rename(ENSGID=Probe)%>%select(ENSGID,Tissue,H4_abf)

In [132]:
# Take maximum coloc score of variant per gene per tissue

In [133]:
gene_variant_2<- data.frame()
for (i in 1:nrow(gene_variant_1) ){
line_q <- gene_variant_1[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(coloc%>%filter(ENSGID==ensg_q))
if (genep>0){
c_df <- coloc%>%filter(ENSGID==ensg_q)%>%select(Tissue,H4_abf)%>%mutate(weight = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
pip_coloc <- max(c_df$H4_abf)
coloc_weight <- max(as.double((c_df%>%filter(H4_abf==pip_coloc))$weight))
}else {
pip_coloc <- 0
coloc_weight <- 0
}
tmp_df <- gene_variant_1[i,]%>%mutate(coloc_pip=pip_coloc,coloc_WT=coloc_weight)
gene_variant_2 <- rbind(gene_variant_2,tmp_df)
}

In [135]:
## Add SMR

In [136]:
smr <- read.table("../../ANALYSIS/SMR//Trans_ancestry_SMR.tab",header=T)%>%rename(ENSGID=probeID)

In [137]:
gene_variant_3 <- data.frame()
for (i in 1:nrow(gene_variant_2) ){
line_q <- gene_variant_2[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(smr%>%filter(ENSGID==ensg_q))
if (genep>0){
smr_df <- smr%>%filter(ENSGID==ensg_q)%>%select(filename,p_SMR)%>%mutate(weight_smr = case_when(grepl("Artery|Adipose|Heart", filename) ~ "1",TRUE ~ '0.8'))
p_smr <- min(smr_df$p_SMR)
smr_weight <- max(as.double((smr_df%>%filter(p_SMR==p_smr))$weight_smr))
}else {
p_smr <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_2[i,]%>%mutate(p_SMR=p_smr,SMR_WT=smr_weight)
gene_variant_3 <- rbind(gene_variant_3,tmp_df)
}


In [139]:
## Add eQTL overlaps

In [140]:
eqtl_overlaps <- read.table("EQTL_OVERLAPS.txt",fill=TRUE,row.names=NULL,header=TRUE)

In [141]:
gene_variant_4 <- data.frame()
for (i in 1:nrow(gene_variant_3) ){
line_q <- gene_variant_3[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(eqtl_overlaps%>%filter(Transcript==ensg_q))
if (genep>0){
smr_df <- eqtl_overlaps%>%filter(Transcript==ensg_q)%>%select(Tissue,P_EQTL)%>%mutate(weight_lookup = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
p_smr <- min(smr_df$P_EQTL)
smr_weight <- max(as.double((smr_df%>%filter(P_EQTL==p_smr))$weight_lookup))
}else {
p_smr <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_3[i,]%>%mutate(P_EQTL=p_smr,EQTL_WT=smr_weight)
gene_variant_4 <- rbind(gene_variant_4,tmp_df)
}


In [144]:
### LDSC(Expression modifier Score)

In [145]:
ldsc <- read.table("ldsc_RESULTS.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [147]:
gene_variant_5 <- data.frame()
for (i in 1:nrow(gene_variant_4) ){
line_q <- gene_variant_4[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(ldsc%>%filter(GENE==ensg_q))
if (genep>0){
ldsc_df <- ldsc%>%filter(GENE==ensg_q)%>%select(Tissue,Enrichment_p)%>%mutate(weight_ldsc = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
p_ldsc <- min(ldsc_df$Enrichment_p)
smr_weight <- max(as.double((ldsc_df%>%filter(Enrichment_p==p_ldsc))$weight_ldsc))
}else {
p_ldsc <- NA
smr_weight <-0 
}
tmp_df <- gene_variant_4[i,]%>%mutate(LDSC_P=p_ldsc,LDSC_WT=smr_weight)
gene_variant_5 <- rbind(gene_variant_5,tmp_df)
}


In [149]:
## Add EMS overlaps

In [150]:
ems <- read.table("OVERLAP_EMS.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [152]:
gene_variant_6 <- data.frame()
for (i in 1:nrow(gene_variant_5) ){
line_q <- gene_variant_5[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(ems%>%filter(GENE==ensg_q))
if (genep>0){
ems_df <- ems%>%filter(GENE==ensg_q)%>%select(Tissue,scaled_EMS)%>%mutate(weight_ems = case_when(grepl("Artery|Adipose|Heart", Tissue) ~ "1",TRUE ~ '0.8'))
s_ems <- max(ems_df$scaled_EMS)
ems_weight <- max(as.double((ems_df%>%filter(scaled_EMS==s_ems))$weight_ems))
}else {
s_ems <- 0
ems_weight <-0 
}
tmp_df <- gene_variant_5[i,]%>%mutate(EMS_SCORE=s_ems,EMS_WT=ems_weight)
gene_variant_6 <- rbind(gene_variant_6,tmp_df)
}

In [154]:
### Add POPS
stand01 <- function(x){(x-min(x,na.rm = TRUE))/(max(x,na.rm = TRUE)-min(x,na.rm = TRUE))}

In [155]:
pops_ems <- read.table("POPS.txt",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(POPS=stand01(PoPS_score))

In [157]:
gene_variant_7 <- data.frame()
for (i in 1:nrow(gene_variant_6) ){
line_q <- gene_variant_6[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(pops_ems%>%filter(ENSG==ensg_q))
if (genep>0){
pops_df <- pops_ems%>%filter(ENSG==ensg_q)%>%select(POPS)
pops_score <- as.double(pops_df)
}else {
pops_score <- 0 
}
tmp_df <- gene_variant_6[i,]%>%mutate(POPS_SCORE=pops_score)
gene_variant_7 <- rbind(gene_variant_7,tmp_df)
}

In [159]:
## Add DOWNSTREAMER

In [160]:
down_df <- read.table("Downstreamer.txt",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(DOWNSTREAMER_P=pnorm(q=ZSCORE, lower.tail=TRUE))

In [162]:
gene_variant_8 <- data.frame()
for (i in 1:nrow(gene_variant_7) ){
line_q <- gene_variant_7[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(down_df%>%filter(ENSG==ensg_q))
if (genep>0){
downs_df <- down_df%>%filter(ENSG==ensg_q)%>%select(DOWNSTREAMER_P)
p_down <- min(downs_df$DOWNSTREAMER_P)
}else {
p_down <- NA
}
tmp_df <- gene_variant_7[i,]%>%mutate(DOWN_P=p_down)
gene_variant_8 <- rbind(gene_variant_8,tmp_df)
}

In [164]:
## Add EpiMAp

In [165]:
epi_map <- read.table("EPIMAP.tab",fill=TRUE,row.names=NULL,header=TRUE)

In [167]:
gene_variant_9 <- data.frame()
for (i in 1:nrow(gene_variant_8) ){
line_q <- gene_variant_8[i,]
ensg_q <- as.character(line_q[1])
genep<- nrow(epi_map%>%filter(gene==ensg_q))
if (genep>0){
epi_df <- epi_map%>%filter(gene==ensg_q)%>%select(score)
epi_score <- max(epi_df$score)
}else {
epi_score <- 0
}
tmp_df <- gene_variant_8[i,]%>%mutate(EPIMAP_SCORE=epi_score)
gene_variant_9 <- rbind(gene_variant_9,tmp_df)
}

In [169]:
## Add Genehancer

In [170]:
gh <- read.table("GENEHANCER.TAB",fill=TRUE,row.names=NULL,header=TRUE)%>%mutate(CHROM=as.integer(om))%>%rename(START=name,END=start,SCORE=end)%>%
select(-om)%>%mutate(SCORE=stand01(SCORE))

Warning message in mask$eval_all_mutate(quo):
"NAs introduced by coercion"


In [172]:
gene_variant_10 <- data.frame()
for (i in 1:nrow(gene_variant_9) ){
line_q <- gene_variant_9[i,]
chr_q <- as.integer(line_q[3])
start_q<- as.integer(line_q[4])
end_q <- as.integer(line_q[5])
genep<- nrow(gh%>%filter(CHROM==chr_q & START >= start_q & END <= end_q))
if (genep>0){
gh_df <- gh%>%filter(CHROM==chr_q & START >= start_q & END <= end_q)%>%select(SCORE)
gh_score <- max(gh_df$SCORE)
}else {
gh_score <- 0
}
tmp_df <- gene_variant_9[i,]%>%mutate(GH_SCORE=gh_score)
gene_variant_10 <- rbind(gene_variant_10,tmp_df)
}

In [175]:
## Add CS2GO SCORE

In [176]:
cs2g <- read.table("TRANS_CS2G.TAB",fill=TRUE,row.names=NULL,header=TRUE)

In [178]:
gene_variant_11 <- data.frame()
for (i in 1:nrow(gene_variant_10) ){
line_q <- gene_variant_10[i,]
ensg_q <- as.character(line_q[2])
genep<- nrow(cs2g%>%filter(GENENAME==ensg_q))
if (genep>0){
cs2g_df <- cs2g%>%filter(GENENAME==ensg_q)%>%select(SCORE)
score <- max(cs2g_df$SCORE)
}else {
score <- NA
}
tmp_df <- gene_variant_10[i,]%>%mutate(CS2G_SCORE=score)
gene_variant_11 <- rbind(gene_variant_11,tmp_df)
}

In [184]:
## CALCULATE GENE PRIORITY SCORE

In [186]:
gene_priority <- data.frame()
for (i in 1:nrow(gene_variant_11) ){
line_q <- gene_variant_11[i,] 
p_g_raw <- na.omit(c(line_q$MAGMA_P,line_q$p_SMR,line_q$P_EQTL,line_q$LDSC_P,line_q$DOWN_P))
if (length(p_g_raw)<2){
p_g <- p_g_raw
} else {
p_g <- -log10(as.double(allmetap(p_g_raw,method="sumz")$p))}
S_G <- sum(as.double(na.omit(c(line_q$coloc_pip,line_q$EMS_SCORE,line_q$POPS_SCORE,line_q$EPIMAP_SCORE,line_q$GH_SCORE,line_q$CS2G_SCORE))))
C_S <- sum(as.double(na.omit(c(line_q$coloc_WT,line_q$SMR_WT,line_q$EQTL_WT,line_q$LDSC_WT,line_q$EMS_WT)))+
           length(as.double(na.omit(line_q$MAGMA_P,line_q$POPS_SCORE,line_q$DOWN_P,line_q$EPIMAP_SCORE,line_q$GH_SCORE,line_q$CS2G_SCORE))))/11
GP_SCORE <- ((as.double(p_g)+as.double(S_G))*as.double(C_S))/log2(line_q$TSS_DISTANCE)
tmp_df <- gene_variant_11[i,]%>%mutate(GP_SCORE=as.double(GP_SCORE),P_G=p_g,SG=S_G,CS=C_S)
gene_priority  <- rbind(gene_priority,tmp_df)
}

In [187]:
gene_priority <- arrange(gene_priority%>%group_by(WINDOW)%>% mutate(mx = max(GP_SCORE)) %>% arrange(desc(mx), desc(GP_SCORE)) %>% select(-mx),CHR,POSITION)%>%
select(NAME,CHR,POSITION,INDEX,GP_SCORE,TSS_DISTANCE,P_G,SG,CS,MAGMA_P,p_SMR,P_EQTL,LDSC_P,DOWN_P,coloc_pip,EMS_SCORE,POPS_SCORE,GH_SCORE,CS2G_SCORE,coloc_WT,SMR_WT,EQTL_WT,LDSC_WT,EMS_WT)

Adding missing grouping variables: `WINDOW`


In [88]:
normalit<-function(m){
   (m - mean(m))/(sd(m))
 }

In [89]:
## Calculate the relative  importance of GPscore Terms

In [ ]:
df <- gene_priority%>%group_by(WINDOW)%>% mutate(CS2G_SCORE = ifelse(is.na(CS2G_SCORE), 0, CS2G_SCORE))%>%mutate(TSS_norm=normalit(log2(TSS_DISTANCE)),PG=normalit(P_G),CSN=normalit(CS),SGN=normalit(SG))
df_csf1 <- df%>%filter(WINDOW==1)
df1 <- df%>%group_by(WINDOW) %>%do( gpcontrib= (calc.relimp(lm(GP_SCORE ~  TSS_norm+ PG + CSN+ SGN, data = .),rela = TRUE))$lmg)